## Exemplary SA-CCR allocation considering TH and MTA

Our goal is to perform an Euler allocation for the minimal example of a one trade portfolio.
We use the same 200Bn IRS as in previous examples.

In [1]:
import QuantLib as ql
from IPython.core.display import display, Markdown
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement
from instruments.interestRateInstrument.irs import IRS
from jupyterUtils import export
from marketdata.interestRateIndices import InterestRateIndex
from marketdata import init_marketdata
from sa_ccr.sa_ccr import SA_CCR
asdf =1

In [2]:
irs = IRS(notional=200000000000,
          index=InterestRateIndex.USDLIBOR3M,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years))

In [3]:
ca = CollateralAgreement(threshold=2000000000,
                         mta=0)
ca.link_sa_ccr_instance(SA_CCR(ca))
ca.add_trades(irs)

The inclusion of the threshold raises the ead since it lowers the available overcollataralization.

In [4]:
ead_with_threshold = ca.get_sa_ccr_model().get_risk_measure()
print(ead_with_threshold)
ca.threshold = 0
ead_no_threshold = ca.get_sa_ccr_model().get_risk_measure()
print(ead_no_threshold)
ca.threshold = 2000000000

817798882.3982577
582881953.4648591


In [5]:
display(Markdown('The EAD with threshold is %.2f while the EAD without threshold is %.2f' %(ead_with_threshold, ead_no_threshold)))

The EAD with threshold is 817798882.40 while the EAD without threshold is 582881953.46

When trying to allocate with threshold we realize that the allocation is not working due to the missing homogeneity of C.

In [6]:
eulerAllocator = EulerAllocator(ca)
allocated_value = eulerAllocator.allocate_ead()[irs]
print(allocated_value)

526604164.2713547


In [7]:
display(Markdown('The value of %.2f that has been allocated to the single trade in the portfolio is far off from the portfolios EAD of %.2f' % (allocated_value, ead_with_threshold)))

The value of 526604164.27 that has been allocated to the single trade in the portfolio is far off from the portfolios EAD of 817798882.40

If we instead allocate without threshold, the allocation works.

In [8]:
ca.threshold = 0
allocated_value=eulerAllocator.allocate_ead()[irs]
print(allocated_value)

582887839.6499157


In [9]:
display(Markdown('The value of %.2f that has been allocated to the single trade in the portfolio equals the portfolios EAD without threshold of %.2f' % (allocated_value, ead_no_threshold)))

The value of 582887839.65 that has been allocated to the single trade in the portfolio equals the portfolios EAD without threshold of 582881953.46

A reasonable approach to allocate an SA-CCR EAD under consideration of a threshold could be to allocate without threshold and then scale accordingly:

\begin{align*}
    X_{t\text{, TH}} = X_{t\text{, no TH}} \frac{EAD_{\text{TH}}}{EAD_{\text{no TH}}}
\end{align*}

In [10]:
print(allocated_value * (ead_with_threshold/ead_no_threshold))

817807140.8724346


In [13]:
export('Exemplary SA-CCR allocation considering MTA and TH.ipynb')